# Training utilizzando il dataset CICIDS18

In [72]:
import sys

SAVE_ATTACKS=True #set to true if you want to save generated attacks
SAVE_TRAIN_TEST=True #set to true if you want to save train test splits

In [73]:
#!conda  install  --yes --prefix {sys.prefix} -c conda-forge numpy==1.20
#!conda  install  --yes --prefix {sys.prefix} -c conda-forge pandas
#!conda  install  --yes --prefix {sys.prefix} -c conda-forge matplotlib
#!conda  install  --yes --prefix {sys.prefix} -c conda-forge xgboost
#!conda  install  --yes --prefix {sys.prefix} -c conda-forge scikit-learn

#SOME SOLUTIONS REQUIRE TENSORFLOW 1.5, WHILE ONE PYOD AUTOENCODER WANTS 2.0. THIS CREATES SOME COMPATIBILITY PROBLEMS.
#!conda  install  --yes --prefix {sys.prefix} -c conda-forge tensorflow==2
#!conda  install  --yes --prefix {sys.prefix} -c conda-forge keras
#!conda  install  --yes --prefix {sys.prefix} -c conda-forge  copulas
#!conda  install  --yes --prefix {sys.prefix} -c conda-forge mkl-service

In [74]:
#!{sys.executable} -m pip install pyod
#!{sys.executable} -m pip install seaborn
#!{sys.executable} -m pip install tabgan
#!{sys.executable} -m pip install sbo

In [75]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [76]:
import sys, os, time, glob, joblib, random, itertools, PIL
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef
from sklearn.manifold import TSNE
from sklearn.ensemble import IsolationForest
from sklearn import ensemble
from sklearn import metrics
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
from matplotlib import cm
from time import time
import tensorflow as tf
#import seaborn as sns
#import pylab as plt
from IPython import display
from itertools import combinations
#necessary to avoid growth of jupyter
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [77]:
PATH='/home/polazzi/datasets/cicids/'
cicids_competitors=open(PATH+"cicids_competitors.csv", "a")

In [78]:
cicids_competitors.write('\ndataset, '
                       + 'generation technique, '
                       + 'classifier, '
                       + 'train-test split, ' 
                       + '# normal in training set, '
                       + '# original attack in training set, '
                       + '# generated attack in training set, '
                       + '# normal in test set, ' 
                       + '# attacks in test set, '
                       + ' features (columns) - may be 1-hotted, '
                       + ' generated attack is manipulated, '
                       + 'tp, tn, fp, fn, accuracy, mcc \n')
cicids_competitors.flush()

In [79]:
# scale and move the coordinates so they fit [0; 1] range
def scale_to_01_range(x):
    # compute the distribution range
    value_range = (np.max(x) - np.min(x))

    # move the distribution so that it starts from zero
    # by extracting the minimal value from all its values
    starts_from_zero = x - np.min(x)

    # make the distribution fit [0; 1] by dividing by its range
    if(value_range!=0):
        return starts_from_zero / value_range
    else:
        return starts_from_zero

### Partizionamento del dataset

Come prima cosa, si crea la partizione di training e di test con un rapporto 60:40.
La colonna contenente le etichette viene normalizzata con ivalori [0,1].

In [80]:
CICIDS=pd.read_csv(PATH+"CICIDS18_Shuffled_Reduced.csv")
CICIDS.replace([np.inf, -np.inf], np.nan, inplace=True)
CICIDS.fillna(-1, inplace=True)
CICIDS_Y = CICIDS[['label']]
print(CICIDS_Y['label'].unique())

CICIDS.info()

CICIDS=CICIDS.drop(columns=["Timestamp"])
CICIDS['label'] =CICIDS['label'].replace(['normal'],0)
CICIDS.loc[CICIDS["label"] !=0, "label"] = 1
CICIDS_X=CICIDS.apply(pd.to_numeric)
CICIDS_Y=CICIDS['label'].apply(pd.to_numeric)
CICIDS_X=CICIDS_X.drop('label', axis=1, inplace=False)

['normal' 'DDOS attack-HOIC' 'SSH-Bruteforce' 'FTP-BruteForce'
 'DoS attacks-Hulk' 'Infilteration' 'Bot' 'DoS attacks-SlowHTTPTest'
 'DDOS attack-LOIC-UDP']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199997 entries, 0 to 199996
Data columns (total 79 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Protocol           199997 non-null  int64  
 1   Timestamp          199997 non-null  object 
 2   Flow_Duration      199997 non-null  int64  
 3   Tot_Fwd_Pkts       199997 non-null  int64  
 4   Tot_Bwd_Pkts       199997 non-null  int64  
 5   TotLen_Fwd_Pkts    199997 non-null  int64  
 6   TotLen_Bwd_Pkts    199997 non-null  int64  
 7   Fwd_Pkt_Len_Max    199997 non-null  int64  
 8   Fwd_Pkt_Len_Min    199997 non-null  int64  
 9   Fwd_Pkt_Len_Mean   199997 non-null  float64
 10  Fwd_Pkt_Len_Std    199997 non-null  float64
 11  Bwd_Pkt_Len_Max    199997 non-null  int64  
 12  Bwd_Pkt_Len_Min    199997 non-null  int64

In [81]:
CICIDS_X

,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,Fwd_Pkt_Len_Mean,Fwd_Pkt_Len_Std,...,Fwd_Act_Data_Pkts,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
0,6,4368867,5,3,935,343,935,0,187.000000,418.144712,...,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6,6157,3,4,289,935,289,0,96.333333,166.854228,...,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,48,2,0,0,0,0,0,0.000000,0.000000,...,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,6,1,1,0,0,0,0,0.000000,0.000000,...,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,2,1,1,0,0,0,0,0.000000,0.000000,...,0,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199992,6,6,1,1,0,0,0,0,0.000000,0.000000,...,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199993,6,1393225,8,7,1132,1581,661,0,141.500000,222.792023,...,5,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199994,6,5277048,5,3,935,364,935,0,187.000000,418.144712,...,1,32,0.0,0.0,0.0,0.0,5205095.0,0.0,5205095.0,5205095.0
199995,6,933,2,0,0,0,0,0,0.000000,0.000000,...,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
CICIDS_Y

0         0
1         1
2         0
3         1
4         1
         ..
199992    1
199993    0
199994    0
199995    1
199996    0
Name: label, Length: 199997, dtype: int64

In [87]:
#Eseguire solo una volta, altrimenti la codifica one-hot non funziona più
#CICIDS_train_X, CICIDS_test_X,CICIDS_train_Y, CICIDS_test_Y = train_test_split(CICIDS_X,CICIDS_Y, 
#                                                                                    test_size=0.4, 
#                                                                                    shuffle=True)
#
#
#CICIDS_train_X.to_pickle(PATH+'/cicids_Preliminary_Train_X.pkl')
#CICIDS_train_Y.to_pickle(PATH+'/cicids_Preliminary_Train_Y.pkl')
#CICIDS_test_X.to_pickle(PATH+'/cicids_Preliminary_Test_X.pkl')
#CICIDS_test_Y.to_pickle(PATH+'/cicids_Preliminary_Test_Y.pkl')

In [88]:
CICIDS_train_X=pd.read_pickle(PATH+'/cicids_Preliminary_Train_X.pkl')
CICIDS_train_Y=pd.read_pickle(PATH+'/cicids_Preliminary_Train_Y.pkl')
CICIDS_test_X=pd.read_pickle(PATH+'/cicids_Preliminary_Test_X.pkl')
CICIDS_test_Y=pd.read_pickle(PATH+'/cicids_Preliminary_Test_Y.pkl')

In [89]:
print('train_size, test_size')
CICIDS_train_X.shape, CICIDS_test_X.shape #,contamination

train_size, test_size


((119998, 77), (79999, 77))

In [90]:
CICIDS_train_X_no_oneHot=CICIDS_train_X
CICIDS_test_X_no_oneHot=CICIDS_test_X
CICIDS_test_Y_no_oneHot=CICIDS_test_Y

In [91]:
type(CICIDS_test_X), type(CICIDS_test_Y)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [92]:
np.any(np.isnan(CICIDS_train_X)), np.any(np.isnan(CICIDS_test_X))

(False, False)

I categorici da mettere in one-hot encoding sono:
- Protocol 3
- Fwd_PSH_Flags 2
- Bwd_PSH_Flags 1
- Fwd_URG_Flags 2
- Bwd_URG_Flags 1
- FIN_Flag_Cnt 2
- SYN_Flag_Cnt 2
- RST_Flag_Cnt 2
- PSH_Flag_Cnt 2
- ACK_Flag_Cnt 2
- URG_Flag_Cnt 2
- CWE_Flag_Count 2
- ECE_Flag_Cnt 2
- DownUp_Ratio 13
- Fwd_Bytsb_Avg 1
- Fwd_Pktsb_Avg 1
- Fwd_Blk_Rate_Avg 1
- Bwd_Bytsb_Avg 1
- Bwd_Pktsb_Avg 1
- Bwd_Blk_Rate_Avg 1

In [93]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')

array_train= pd.DataFrame().assign(A=CICIDS_train_X['Protocol'],
                             B=CICIDS_train_X['Fwd_PSH_Flags'],
                             C=CICIDS_train_X['Bwd_PSH_Flags'], 
                             D=CICIDS_train_X['Fwd_URG_Flags'],
                             E=CICIDS_train_X['Bwd_URG_Flags'],
                             F=CICIDS_train_X['FIN_Flag_Cnt'],
                             G=CICIDS_train_X['SYN_Flag_Cnt'],
                             H=CICIDS_train_X['RST_Flag_Cnt'],
                             I=CICIDS_train_X['PSH_Flag_Cnt'],
                             J=CICIDS_train_X['ACK_Flag_Cnt'],
                             K=CICIDS_train_X['URG_Flag_Cnt'],
                             L=CICIDS_train_X['CWE_Flag_Count'],
                             M=CICIDS_train_X['ECE_Flag_Cnt'],
                             N=CICIDS_train_X['DownUp_Ratio'],
                             P=CICIDS_train_X['Fwd_Bytsb_Avg'],
                             Q=CICIDS_train_X['Fwd_Pktsb_Avg'],
                             R=CICIDS_train_X['Fwd_Blk_Rate_Avg'],
                             S=CICIDS_train_X['Bwd_Bytsb_Avg'],
                             T=CICIDS_train_X['Bwd_Pktsb_Avg'],
                             U=CICIDS_train_X['Bwd_Blk_Rate_Avg'],
                            )
array_train.shape

(119998, 20)

In [94]:
#enc=enc.fit(array)
enc.fit(array_train)

encoded=enc.transform(array_train).toarray()
encoded.shape, array_train.shape

((119998, 42), (119998, 20))

In [95]:
CICIDS_train_X = CICIDS_train_X.join(pd.DataFrame(encoded))
CICIDS_train_X=CICIDS_train_X.fillna(0, inplace= False)
CICIDS_train_X.shape

(119998, 119)

In [96]:
np.any(np.isnan(encoded)), np.any(np.isnan(array_train))

(False, False)

In [97]:
array_test= pd.DataFrame().assign(
                             A=CICIDS_test_X['Protocol'],
                             B=CICIDS_test_X['Fwd_PSH_Flags'],
                             C=CICIDS_test_X['Bwd_PSH_Flags'], 
                             D=CICIDS_test_X['Fwd_URG_Flags'],
                             E=CICIDS_test_X['Bwd_URG_Flags'],
                             F=CICIDS_test_X['FIN_Flag_Cnt'],
                             G=CICIDS_test_X['SYN_Flag_Cnt'],
                             H=CICIDS_test_X['RST_Flag_Cnt'],
                             I=CICIDS_test_X['PSH_Flag_Cnt'],
                             J=CICIDS_test_X['ACK_Flag_Cnt'],
                             K=CICIDS_test_X['URG_Flag_Cnt'],
                             L=CICIDS_test_X['CWE_Flag_Count'],
                             M=CICIDS_test_X['ECE_Flag_Cnt'],
                             N=CICIDS_test_X['DownUp_Ratio'],
                             P=CICIDS_test_X['Fwd_Bytsb_Avg'],
                             Q=CICIDS_test_X['Fwd_Pktsb_Avg'],
                             R=CICIDS_test_X['Fwd_Blk_Rate_Avg'],
                             S=CICIDS_test_X['Bwd_Bytsb_Avg'],
                             T=CICIDS_test_X['Bwd_Pktsb_Avg'],
                             U=CICIDS_test_X['Bwd_Blk_Rate_Avg'],
                            )
array_train.shape

(119998, 20)

In [98]:
encoded=enc.transform(array_test).toarray()
encoded.shape, array_test.shape, CICIDS_test_X.shape

((79999, 42), (79999, 20), (79999, 77))

In [99]:
CICIDS_test_X = CICIDS_test_X.join(pd.DataFrame(encoded))
CICIDS_test_X=CICIDS_test_X.fillna(0, inplace= False)

CICIDS_test_X.shape

(79999, 119)

Proviamo XGBoost in un ambiente supervisionato.

In [100]:
xb=xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False)
xb.fit(CICIDS_train_X, CICIDS_train_Y)
predicted=xb.predict(CICIDS_test_X)

In [101]:
accuracy=accuracy_score(CICIDS_test_Y, predicted)
tn, fp, fn, tp = confusion_matrix(CICIDS_test_Y, predicted).ravel()
mcc=matthews_corrcoef(CICIDS_test_Y, predicted)
print("tp, tn, fp, fn, accuracy, mcc:")
tp, tn, fp, fn, accuracy, mcc

tp, tn, fp, fn, accuracy, mcc:


(50487, 26872, 303, 2337, 0.9669995874948437, 0.9291909071273011)

In [102]:
normal_in_training_set=np.unique(CICIDS_train_Y, return_counts=True)[1][0]
original_attack_in_training_set=np.unique(CICIDS_train_Y, return_counts=True)[1][1]
normal_in_test_set=np.unique(CICIDS_test_Y, return_counts=True)[1][0]
attacks_in_test_set=np.unique(CICIDS_test_Y, return_counts=True)[1][1]

cicids_competitors.write('CICIDS, '
                       + 'all known, '
                       + 'XGBoost, '
                       + '0.6--0.4, '
                       + str(normal_in_training_set) + ', '
                       + str(original_attack_in_training_set) + ', ' + '0, '
                       + str(normal_in_test_set) + ', '
                       + str(attacks_in_test_set) + ', '
                       + str(CICIDS_test_X.shape[1]) + ', '
                       + ' NO GENERATED ATTACK, '
                       + f'{tp}, {tn}, {fp}, {fn}, {accuracy:.4f}, {mcc:.4f}\n')
cicids_competitors.flush()

**Adesso supponiamo di non avere conoscenza degli attacchi. Questo vuol dire che non c'è nessun attacco nel training set.**

contamination è impostata a 0 (o a un valore di default se 0 non è accettato come valore), gli attacchi sono rimossi dal training set, e quindi questi si trovano solo nel test set d'ora in poi.

In [103]:
trainingX_Y=pd.concat([CICIDS_train_X, pd.DataFrame(CICIDS_train_Y, columns = ['label'])], axis=1)
CICIDS_train_X_no_attack=trainingX_Y[trainingX_Y.label == 0]

In [104]:
CICIDS_train_X_no_attack=CICIDS_train_X_no_attack.drop(['label'], axis=1)
CICIDS_train_X_no_attack.shape, np.any(np.isnan(CICIDS_train_X_no_attack))

((40319, 119), False)

In [105]:
missing_attacks=trainingX_Y[trainingX_Y['label']!=0]
missing_attacks=missing_attacks.drop(['label'], axis=1)

frames=[CICIDS_test_X, missing_attacks]
CICIDS_test_X=pd.concat(frames).to_numpy()
CICIDS_test_Y=np.hstack((CICIDS_test_Y.to_numpy(), np.ones((missing_attacks.to_numpy().shape[0]))))

In [106]:
CICIDS_test_Y

array([1., 0., 0., ..., 1., 1., 1.])

In [107]:
CICIDS_test_Y.shape,CICIDS_test_X.shape

((159678,), (159678, 119))

In [108]:
CICIDS_train_X_no_attack

,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,Fwd_Pkt_Len_Mean,Fwd_Pkt_Len_Std,...,32,33,34,35,36,37,38,39,40,41
3727,6,3503945,5,3,935,326,935,0,187.000000,418.144712,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
54381,6,117842138,22,20,2766,1205,539,0,125.727273,227.322339,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
33612,17,360,1,1,36,68,36,36,36.000000,0.000000,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
109677,17,324,1,1,36,164,36,36,36.000000,0.000000,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
34450,6,4994533,5,3,935,318,935,0,187.000000,418.144712,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55577,6,4383528,5,4,935,666,935,0,187.000000,418.144712,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
138054,6,15,1,1,0,0,0,0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46580,6,4403468,5,3,935,302,935,0,187.000000,418.144712,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
61919,6,4539635,5,3,935,341,935,0,187.000000,418.144712,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [109]:
#save data in one hot encoding
np.save(PATH+'/train_X_no_attack', CICIDS_train_X_no_attack )
np.save(PATH+'/test_X', CICIDS_test_X)
np.save(PATH+'/test_Y', CICIDS_test_Y )

In [110]:
trainingX_Y=pd.concat([CICIDS_train_X_no_oneHot, pd.DataFrame(CICIDS_train_Y, columns = ['label'])], axis=1)
CICIDS_train_X_no_oneHot=trainingX_Y[trainingX_Y.label == 0]

CICIDS_train_X_no_oneHot=CICIDS_train_X_no_oneHot.drop(['label'], axis=1)
CICIDS_train_X_no_attack.shape

missing_attacks=trainingX_Y[trainingX_Y['label']!=0]
missing_attacks=missing_attacks.drop(['label'], axis=1)
frames=[CICIDS_test_X_no_oneHot, missing_attacks]
CICIDS_test_X_no_oneHot=pd.concat(frames)
CICIDS_test_Y_no_oneHot=np.hstack((CICIDS_test_Y_no_oneHot.to_numpy(), np.ones((missing_attacks.to_numpy().shape[0]))))

#save data WITHOUT one hot encoding (needed for ARN)
CICIDS_train_X_no_oneHot.to_pickle(PATH+'/train_X_no_attack_no_one_hot_encoding.pkl')
np.save(PATH+'/train_Y_no_attack_no_one_hot_encoding',np.zeros((CICIDS_train_X_no_oneHot.shape[0],)))

CICIDS_test_X_no_oneHot.to_pickle(PATH+'/test_X_no_one_hot_encoding.pkl' )
np.save(PATH+'/test_Y_no_one_hot_encoding', CICIDS_test_Y_no_oneHot )

In [111]:
np.unique(CICIDS_test_Y_no_oneHot)

array([0., 1.])

In [112]:
CICIDS_test_Y_no_oneHot.shape, CICIDS_test_X_no_oneHot.shape

((159678,), (159678, 77))

### ALAD
Adesso utilizziamo **ALAD (eGAN)**.

I dati generati si trovano in: '/home/polazzi/eGAN/Efficient-GAN-Anomaly-Detection/data/cicids_generated_attacks.npy'

In [113]:
generated_attacks=np.load('/home/polazzi/eGAN/Efficient-GAN-Anomaly-Detection/data/cicids_generated_attacks.npy')

In [114]:
CICIDS_train_X_no_attack=np.load(PATH+'/train_X_no_attack.npy')
CICIDS_test_X=np.load(PATH+'/test_X.npy')
CICIDS_test_Y=np.load(PATH+'/test_Y.npy')

In [115]:
np.any(np.isnan(generated_attacks)), np.any(np.isnan(CICIDS_train_X_no_attack))

(False, False)

In [116]:
CICIDS_train_X_no_attack.shape, CICIDS_test_X.shape, generated_attacks.shape

((40319, 119), (159678, 119), (50000, 119))

First, we use the generated attacks without any kind of optimization or improvement of data quality.

In [117]:
X_ood=generated_attacks
X_ood.shape

(50000, 119)

In [118]:
CICIDS_train_X__no_attacks_augmented=np.vstack((CICIDS_train_X_no_attack, X_ood))
CICIDS_train_Y__no_attacks_augmented=np.vstack((np.zeros((CICIDS_train_X_no_attack.shape[0],1)),
                                                 np.ones((X_ood.shape[0], 1))))

In [119]:
xb=xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False)

xb.fit(CICIDS_train_X__no_attacks_augmented, CICIDS_train_Y__no_attacks_augmented)
predicted=xb.predict(CICIDS_test_X)

accuracy=accuracy_score(CICIDS_test_Y, predicted)
tn, fp, fn, tp = confusion_matrix(CICIDS_test_Y, predicted).ravel()
mcc=matthews_corrcoef(CICIDS_test_Y, predicted)

print(f'tp, tn, fp, fn, accuracy, mcc:\n{tp}, {tn}, {fp}, {fn}, {accuracy}, {mcc}')

tp, tn, fp, fn, accuracy, mcc:
0, 27175, 0, 132503, 0.1701862498277784, 0.0


In [120]:
normal_in_training_set=np.unique(CICIDS_train_X_no_attack.shape[0])
original_attack_in_training_set=0
normal_in_test_set=np.unique(CICIDS_test_Y, return_counts=True)[1][0]
attacks_in_test_set=np.unique(CICIDS_test_Y, return_counts=True)[1][1]
generated_attacks=X_ood.shape[0]

cicids_competitors.write('CICIDS, '
                       + 'ALAD (eGAN) '
                       + '- 500 epochs, '
                       + 'XGBoost, '
                       + '0.6--0.4, '
                       + str(normal_in_training_set) + ', '
                       + str(original_attack_in_training_set) + ', '
                       + str(generated_attacks) + ', '
                       + str(normal_in_test_set) + ', '
                       + str(attacks_in_test_set) + ', '
                       + str(CICIDS_test_X.shape[1]) + ', '
                       + ' NO AUGMENTATION, '
                       + f'{tp}, {tn}, {fp}, {fn}, {accuracy:.4f}, {mcc:.4f}\n')
cicids_competitors.flush()

In [121]:
contamination=0.2

total=int(CICIDS_train_X_no_attack.shape[0]*contamination)
if(total > X_ood.shape[0]):
    print("contamination is larger than the available attacks.")
    sys.exit(0)

idx = np.random.randint(X_ood.shape[0], size=total)
attacks=X_ood[idx,:]
CICIDS_train_X__no_attacks_augmented=np.vstack((CICIDS_train_X_no_attack, attacks))
CICIDS_train_Y__no_attacks_augmented=np.vstack((np.zeros((CICIDS_train_X_no_attack.shape[0], 1)),
                                                 np.ones((attacks.shape[0],1))))
                                                 
contamination=contamination/(1+contamination)
                                                 
contamination, attacks.shape

(0.16666666666666669, (8063, 119))

In [122]:
CICIDS_train_X__no_attacks_augmented[0]

array([6.00000000e+00, 3.50394500e+06, 5.00000000e+00, 3.00000000e+00,
       9.35000000e+02, 3.26000000e+02, 9.35000000e+02, 0.00000000e+00,
       1.87000000e+02, 4.18144712e+02, 3.26000000e+02, 0.00000000e+00,
       1.08666667e+02, 1.88216188e+02, 3.59880078e+02, 2.28314086e+00,
       5.00563571e+05, 1.27608149e+06, 3.39377000e+06, 4.00000000e+00,
       3.50394500e+06, 8.75986250e+05, 1.67878167e+06, 3.39377600e+06,
       3.20000000e+01, 7.16640000e+04, 3.58320000e+04, 5.02795348e+04,
       7.13850000e+04, 2.79000000e+02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.68000000e+02, 1.04000000e+02,
       1.42696304e+00, 8.56177822e-01, 0.00000000e+00, 9.35000000e+02,
       1.40111111e+02, 3.16982036e+02, 1.00477611e+05, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.57625000e+02, 1.87000000e+02, 1.08666667e+02, 0.00000000e+00,
      

In [124]:
isol_1=IsolationForest(n_estimators=800,
                         max_samples=400,
                         contamination=contamination,
                         max_features=1.0,
                         bootstrap=False,
                         n_jobs=12,warm_start=False).fit(CICIDS_train_X__no_attacks_augmented)

predicted=isol_1.predict(CICIDS_test_X)

Y=CICIDS_test_Y
predicted=np.where(predicted==-1, 1, 0)
accuracy=accuracy_score(Y, predicted)
mcc=matthews_corrcoef(Y, predicted)
tn, fp, fn, tp= confusion_matrix(Y, predicted).ravel()

print(f'tp, tn, fp, fn, accuracy, mcc:\n{tp}, {tn}, {fp}, {fn}, {accuracy}, {mcc}')

tp, tn, fp, fn, accuracy, mcc:
43815, 22702, 4473, 88688, 0.41656959631257906, 0.1358786383614198


In [42]:
normal_in_training_set=np.unique(CICIDS_train_X_no_attack.shape[0])
original_attack_in_training_set=0
normal_in_test_set=np.unique(CICIDS_test_Y, return_counts=True)[1][0]
attacks_in_test_set=np.unique(CICIDS_test_Y, return_counts=True)[1][1]
generated_attacks=X_ood.shape[0]

cicids_competitors.write('CICIDS, '
                       + 'ALAD (eGAN) '
                       + '- 500 epochs, '
                       + 'Isolation Forest, '
                       + '0.6--0.4, '
                       + str(normal_in_training_set) + ', '
                       + str(original_attack_in_training_set) + ', '
                       + str(generated_attacks) + ', '
                       + str(normal_in_test_set) + ', '
                       + str(attacks_in_test_set) + ', '
                       + str(CICIDS_test_X.shape[1]) + ', '
                       + ' NO AUGMENTATION, '
                       + f'{tp}, {tn}, {fp}, {fn}, {accuracy:.4f}, {mcc:.4f}\n')

cicids_competitors.flush()

### ARN - Adversarial Regularized Reconstruction for Anomaly Detection and Generation

To generate attacks and get results from ARN, execute the notebook: './arn/GENERATION_KDDCUP99REV/CICDS-ARN_CICIDS_REV_Generation.ipynb'

It saves the file with the created data in '/home/polazzi/arn/CICIDS/generated_attacks.npy'

With respect to the other, there is a mandatory step of one-hot encoding and normalization of data. This seems to help a lot the algorithm (but not other algorithms as XGBoost, apparently).

Ha bisogno anche di creare un train e test set nello stesso modo. Per questa ragione, importiamo lo stesso meotodo di creazione usato in ARN.

In [16]:
generated_attacks=np.load('/home/polazzi/arn/cicids/generated_attacks.npy')

In [17]:
x_train=np.load(PATH+'/ARN_x_train.npy')
x_test=np.load(PATH+'/ARN_x_test.npy')
y_train=np.load(PATH+'/ARN_y_train.npy')
y_test=np.load(PATH+'/ARN_y_test.npy')

In [18]:
x_train.shape, x_test.shape

((40319, 101), (159678, 101))

In [19]:
pd.read_pickle('/home/polazzi/datasets/cicids/test_X_no_one_hot_encoding.pkl').shape

(159678, 77)

In [20]:
x_train_augmented=np.vstack((x_train,generated_attacks))

y_train_augmented=np.hstack((y_train,
                             np.ones(generated_attacks.shape[0],)))

In [21]:
y_train_augmented

array([0., 0., 0., ..., 1., 1., 1.])

In [22]:
xb=xgb.XGBClassifier(objective='binary:logistic', 
                     use_label_encoder=False )

xb.fit(x_train_augmented, 
       y_train_augmented)

predicted=xb.predict(x_test)

accuracy=accuracy_score(y_test, predicted)
tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
mcc=matthews_corrcoef(y_test, predicted)

print(f'tp, tn, fp, fn, accuracy, mcc:\n{tp}, {tn}, {fp}, {fn}, {accuracy}, {mcc}')

tp, tn, fp, fn, accuracy, mcc:
0, 27175, 0, 132503, 0.1701862498277784, 0.0


In [25]:
normal_in_training_set=np.unique(x_train.shape[0])
original_attack_in_training_set=0
normal_in_test_set=np.unique(y_test, return_counts=True)[1][0]
attacks_in_test_set=np.unique(y_test, return_counts=True)[1][1]
ga=generated_attacks.shape[0]

cicids_competitors.write('CICIDS, '
                       + 'ARN '
                       + '- 200 epochs, '
                       + 'XGBoost, '
                       + '0.6--0.4, '
                       + str(normal_in_training_set) + ', '
                       + str(original_attack_in_training_set) + ', '
                       + str(ga) + ', '
                       + str(normal_in_test_set) + ', '
                       + str(attacks_in_test_set) + ', '
                       + str(CICIDS_test_X.shape[1]) + ', '
                       + ' NO AUGMENTATION, '
                       + f'{tp}, {tn}, {fp}, {fn}, {accuracy:.4f}, {mcc:.4f}\n')
cicids_competitors.flush()

NameError: name 'CICIDS_test_X' is not defined

In [23]:
contamination=0.2

total=int(x_train.shape[0]*contamination)
if(total > generated_attacks.shape[0]):
    print("contamination is larger than the available attacks.")
    sys.exit(0)
    
np.random.shuffle(generated_attacks)

idx = np.random.randint(generated_attacks.shape[0], size=total)
attacks=generated_attacks[idx,:]
x_train_augmented=np.vstack((x_train, attacks))
y_train_augmented=np.hstack((y_train,np.ones((attacks.shape[0],))))
                                                 
contamination=contamination/(1+contamination)
                                                 
contamination, attacks.shape

(0.16666666666666669, (8063, 101))

In [24]:
isol_1=IsolationForest(n_estimators=800,
                         max_samples= 400,
                         contamination=contamination,
                         max_features=1.0,
                         bootstrap=False,
                         n_jobs=12,warm_start=False).fit(x_train_augmented)

predicted=isol_1.predict(x_test)

Y=y_test
predicted=np.where(predicted==-1, 1, 0)
accuracy=accuracy_score(Y, predicted)
mcc=matthews_corrcoef(Y, predicted)
tn, fp, fn, tp= confusion_matrix(Y, predicted).ravel()

print(f'tp, \ttn, \tfp, \tfn, \taccuracy, \tmcc:\n{tp}, \t{tn}, \t{fp}, \t{fn}, \t{accuracy}, \t{mcc}')

tp, 	tn, 	fp, 	fn, 	accuracy, 	mcc:
64256, 	21976, 	5199, 	68247, 	0.5400368241085184, 	0.22257655115310696


In [53]:
normal_in_training_set=np.unique(x_train.shape[0])
original_attack_in_training_set=0
normal_in_test_set=np.unique(y_test, return_counts=True)[1][0]
attacks_in_test_set=np.unique(y_test, return_counts=True)[1][1]
ga=generated_attacks.shape[0]

cicids_competitors.write('CICIDS, '
                       + 'ARN '
                       + '- 200 epochs, '
                       + 'Isolation Forest, '
                       + '0.6--0.4, '
                       + str(normal_in_training_set) + ', '
                       + str(original_attack_in_training_set) + ', '
                       + str(ga) + ', '
                       + str(normal_in_test_set) + ', '
                       + str(attacks_in_test_set) + ', '
                       + str(CICIDS_test_X.shape[1]) + ', '
                       + ' NO AUGMENTATION, '
                       + f'{tp}, {tn}, {fp}, {fn}, {accuracy:.4f}, {mcc:.4f}\n')
cicids_competitors.flush()